In [1]:
import autograd
import autograd.numpy as np
import matplotlib.pyplot as plt
from PIL import Image 
import os
import IPython.display
import time
from astropy.nddata.utils import Cutout2D
from scipy import ndimage
import pickle
import time
from scipy import signal
from joblib import Parallel, delayed # parallelize for loop
import multiprocessing # find number of cpu threads available
from tqdm.notebook import tqdm # progress bar

In [3]:
#extraction stuff
#Basic data structure for holding bubble information
class BubbleEvent:
    def __init__(self, File):
        #temp pixel arrays and event level meta data
        self.FileName = File
        Bot1PixelArray, Bot2PixelArray = BubbleEvent.GetPixelArray(self.FileName) #gets 2d array of pixel intensities
        self.Date, self.Run, self.EventID = BubbleEvent.GetRunInfo(self.FileName) #parses image name to get event info
        self.BubbleCount = 0
        #actual features to use to classify
        self.UsefulEdgeFeature0, self.UsefulEdgeFeature1, self.UsefulEdgeFeature2 = (GetEdgeFeature(
                                        DownSampleTheArray(2, Bot1PixelArray)) + 
                                        GetEdgeFeature(DownSampleTheArray(2, Bot2PixelArray))) #edge detect. sum
        self.UsefulBlobFeature = np.std(GetBlobs(Bot1PixelArray)) + np.std(GetBlobs(Bot2PixelArray)) #blob convalution deviation
        self.CountBlobPeakFeature = GetPeaks(Bot1PixelArray) + GetPeaks(Bot2PixelArray)
    def GetPixelArray(FileName):
        im = Image.open(FileName)
        PixelArray = np.asarray(im)
        Cutout = Cutout2D(PixelArray, (530,140), 235) #just cut out the parts of the image with bottles
        Bot1PixelArray = Cutout.data
        PixelArray =ndimage.rotate(PixelArray, -45)
        Cutout2 = Cutout2D(PixelArray, (270,310), 235) #other bottle view
        Bot2PixelArray = Cutout2.data
        return Bot1PixelArray, Bot2PixelArray
    def GetRunInfo(File):
        Date = int(File.split("/")[-1].split("_")[0]) #file should be date_run_event
        Run = int(File.split("/")[-1].split("_")[1])
        Event = int("{}{}{}".format(Date, Run,File.split("/")[-1].split("_")[2])) 
        return Date, Run, Event
#Functions that extract useful features from a pixel array
def Convolve(PixelArray, Kernel): #Convolve a given kernel with an array
    Convalution = signal.convolve2d(PixelArray, Kernel, mode='valid')
    return Convalution
#kernel for 
def LaplaceOfGaussKernel(x, y, sigma):
    PointsX, PointsY = np.meshgrid(x,y)
    r = PointsX**2 + PointsY**2
    LoG = -1/(np.pi*sigma**4)*(1 - r/(2*sigma**2))*np.exp(-r/(2*sigma**2))
    return LoG*100

def GetBlobs(PixelArray): #run blob detection and then summarize result
    I = np.arange(-5,6)
    J = np.arange(-5,6)
    Kernel=LaplaceOfGaussKernel(I, J, 8.3)
    Convalution = Convolve(PixelArray, Kernel)
    return Convalution

def GetPeaks(Array):
    Convalution = GetBlobs(Array)
    Peaks = FindPeaks(Convalution, 15)
    return Peaks

def FindPeaks(Array, boxsize): #finds the number of elements that are bigger than all neighbors in some box
    FoundPeak = False
    PeakCount=0
    for i in range(boxsize, np.shape(Array)[0]-boxsize):
        for j in range(boxsize, np.shape(Array)[1]-boxsize):
            CurrentElem = Array[i,j]
            BoxElements = Array[i-boxsize:i+boxsize+1, j-boxsize:j+boxsize]
            if(np.max(BoxElements)<=CurrentElem and CurrentElem>np.mean(Array)+3):
                FoundPeak=True
            if(FoundPeak):
                PeakCount=PeakCount+1
                FoundPeak = False #reset the bool
    return PeakCount

def GetEdgeFeature(PixelArray): #edge detection kernel. Can be shortened by rewriting it to use convolve func
    HorizontalKernal = np.array([[1,0,-1],[2,0,-2], [1,0,-1]])
    VerticalKernal = HorizontalKernal.T
    EdgeArray = np.zeros(3) 
    Step = 3
    i=0
    j=0
    Significant = 35
    XConvalution = Convolve(PixelArray, HorizontalKernal)
    YConvalution = Convolve(PixelArray, VerticalKernal)
    Cut = (XConvalution>35)*(YConvalution<35)
    EdgeArray[0] = Cut.sum()
    Cut = (XConvalution<35)*(YConvalution>35)
    EdgeArray[1] = Cut.sum()
    Cut = (XConvalution>35)*(YConvalution>35)
    EdgeArray[2] = Cut.sum()
    return EdgeArray

def DownSampleArray(BoxSize, Array):
    TempArray = Array[:np.shape(Array)[0]-np.shape(Array)[0]%BoxSize, 
                      :np.shape(Array)[1]-np.shape(Array)[1]%BoxSize] #cut away last rows/columns if needed
    Dim = np.shape(TempArray)[0]
    NewArray = TempArray.reshape(Dim//BoxSize, BoxSize, Dim//BoxSize, BoxSize).mean(3).mean(1)
    return NewArray

def GetAllDiffImages():
    path = "../bubbleimages/difffirst/"
    Files = np.array(["{}{}".format(path,file) for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))])
    return Files

def GetBubbleCount(Event, BubbleInfo):
    EventID = Event.EventID
    Cut = BubbleInfo[:,0]==EventID
    if(Cut.sum()!=1): #some events have 2 different bubble count entries
        Event.BubbleCount=9999
    else:
        Event.BubbleCount = int(BubbleInfo[Cut,1])

In [4]:
BubbleCountInfo = np.genfromtxt("bubnumnew.csv", delimiter=",")
DiffImages = GetAllDiffImages()

In [7]:
N_cpu = multiprocessing.cpu_count()
N_cpu

64

In [23]:
def process_event(i):
    Event = BubbleEvent(DiffImages[i])
    if(i%int(len(DiffImages)/10)==0):
        print("On Event {} of {}".format(i, len(DiffImages)))
    GetBubbleCount(Event, BubbleCountInfo)
    if(Event.BubbleCount == 9999):
        SkippedEventCount = 1
    else:
        SkippedEventCount = 0
    return Event, SkippedEventCount

In [24]:
Events_and_skipped = Parallel(n_jobs=N_cpu)(delayed(process_event)(i)\
                                            for i in tqdm(range(0, len(DiffImages)), desc='Event #'))

In [43]:
Events = np.asarray([i[0] for i in Events_and_skipped], dtype=BubbleEvent)
Skipped = np.array([i[1] for i in Events_and_skipped])

In [44]:
Skipped.sum()

140

In [45]:
Events = Events[Skipped == 0]

In [47]:
pickle.dump(Events, open("Events.p", "wb"))